In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv("titanic_train.csv")

In [21]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [22]:
y = data.iloc[:,1].values
print(pd.DataFrame(y))

     0
0    0
1    1
2    1
3    1
4    0
..  ..
884  0
885  1
886  0
887  1
888  0

[889 rows x 1 columns]


In [ ]:
# First I start by removing features that are not going to help in the prediction such as the name of the person

In [23]:
x_train = data.drop('Name',axis=1)
x_train = x_train.drop('PassengerId',axis=1)
x_train = x_train.drop('Age',axis=1)
x_train = x_train.drop('Ticket',axis=1)
x_train = x_train.iloc[:, x_train.columns!='Survived'].values
print(pd.DataFrame(x_train))

     0       1  2  3        4  5
0    3    male  1  0     7.25  S
1    1  female  1  0  71.2833  C
2    3  female  0  0    7.925  S
3    1  female  1  0     53.1  S
4    3    male  0  0     8.05  S
..  ..     ... .. ..      ... ..
884  2    male  0  0       13  S
885  1  female  0  0       30  S
886  3  female  1  2    23.45  S
887  1    male  0  0       30  C
888  3    male  0  0     7.75  Q

[889 rows x 6 columns]


In [24]:
x_test = pd.read_csv("test.csv")
x_test = x_test.drop('PassengerId',axis=1)
x_test = x_test.drop('Name',axis=1)
x_test = x_test.drop('Cabin',axis=1)
x_test = x_test.drop('Age',axis=1)
x_test = x_test.iloc[:, x_test.columns!='Ticket'].values
x_test[152][4] = 83.1583
print(pd.DataFrame(x_test))

     0       1  2  3        4  5
0    3    male  0  0   7.8292  Q
1    3  female  1  0        7  S
2    2    male  0  0   9.6875  Q
3    3    male  0  0   8.6625  S
4    3  female  1  1  12.2875  S
..  ..     ... .. ..      ... ..
413  3    male  0  0     8.05  S
414  1  female  0  0    108.9  C
415  3    male  0  0     7.25  S
416  3    male  0  0     8.05  S
417  3    male  1  1  22.3583  C

[418 rows x 6 columns]


In [ ]:
# I then encode the categorical data which can't be used in models that require numerical values. For example this code
# will turn male to 1 and female to 0 so as to have only numerical data.

In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoder = LabelEncoder()
x_train[:,1] = encoder.fit_transform(x_train[:,1])
encoder2 = LabelEncoder()
x_train[:,5] = encoder2.fit_transform(x_train[:,5])
x_test[:,1] = encoder.fit_transform(x_test[:,1])
x_test[:,5] = encoder2.fit_transform(x_test[:,5])
print(pd.DataFrame(x_train))

     0  1  2  3        4  5
0    3  1  1  0     7.25  2
1    1  0  1  0  71.2833  0
2    3  0  0  0    7.925  2
3    1  0  1  0     53.1  2
4    3  1  0  0     8.05  2
..  .. .. .. ..      ... ..
884  2  1  0  0       13  2
885  1  0  0  0       30  2
886  3  0  1  2    23.45  2
887  1  1  0  0       30  0
888  3  1  0  0     7.75  1

[889 rows x 6 columns]


In [ ]:
# Now I will scale my numerical data so that any data that has very large numbers doesn't negatively impact the overall 
# prediction becuase of the magnitude.

In [26]:
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)
print(pd.DataFrame(x_train))

            0         1         2         3         4         5
0    0.825209  0.735342  0.431350 -0.474326 -0.500240  0.586840
1   -1.572211 -1.359911  0.431350 -0.474326  0.788947 -1.939555
2    0.825209 -1.359911 -0.475199 -0.474326 -0.486650  0.586840
3   -1.572211 -1.359911  0.431350 -0.474326  0.422861  0.586840
4    0.825209  0.735342 -0.475199 -0.474326 -0.484133  0.586840
..        ...       ...       ...       ...       ...       ...
884 -0.373501  0.735342 -0.475199 -0.474326 -0.384475  0.586840
885 -1.572211 -1.359911 -0.475199 -0.474326 -0.042213  0.586840
886  0.825209 -1.359911  0.431350  2.006119 -0.174084  0.586840
887 -1.572211  0.735342 -0.475199 -0.474326 -0.042213 -1.939555
888  0.825209  0.735342 -0.475199 -0.474326 -0.490173 -0.676357

[889 rows x 6 columns]


In [ ]:
# This is a neural network that has an input layer, one hidden layer, and the output layer. I use the relu activation
# function from the input to hidden layer and use the sigmoid activation function for the ouput layer. I use binary 
# cross-entropy in order to calculate the error from the output layer

In [27]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

opt = SGD(lr=0.001)

classifier = Sequential()

classifier.add(Dense(units=5,
                     activation='relu',
                     use_bias=True,
                     kernel_initializer='RandomNormal',
                     bias_initializer='RandomNormal',
                     input_dim=6))

classifier.add(Dense(units=1,
                     activation='sigmoid',
                     use_bias=True,
                     kernel_initializer='RandomNormal',
                     bias_initializer='RandomNormal'))

classifier.compile(optimizer=opt,
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

classifier.fit(x_train, y,
               batch_size=1,
               epochs=120)

neural_network_pred = classifier.predict(x_test)
for i in range(len(neural_network_pred)):
    if neural_network_pred[i] >= 0.5:
        neural_network_pred[i] = 1
    else:
        neural_network_pred[i] = 0

Epoch 1/120
889/889 [==============================] - 0s 551us/step - loss: 0.6872 - accuracy: 0.6232
Epoch 2/120
889/889 [==============================] - 0s 466us/step - loss: 0.6791 - accuracy: 0.6175
Epoch 3/120
889/889 [==============================] - 0s 461us/step - loss: 0.6737 - accuracy: 0.6175
Epoch 4/120
889/889 [==============================] - 0s 496us/step - loss: 0.6698 - accuracy: 0.6175
Epoch 5/120
889/889 [==============================] - 0s 458us/step - loss: 0.6669 - accuracy: 0.6175
Epoch 6/120
889/889 [==============================] - 0s 461us/step - loss: 0.6643 - accuracy: 0.6175
Epoch 7/120
889/889 [==============================] - 0s 458us/step - loss: 0.6619 - accuracy: 0.6175
Epoch 8/120
889/889 [==============================] - 0s 461us/step - loss: 0.6589 - accuracy: 0.6175
Epoch 9/120
889/889 [==============================] - 0s 535us/step - loss: 0.6551 - accuracy: 0.6175
Epoch 10/120
889/889 [==============================] - 0s 465us/step - l

In [29]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
cl = RandomForestClassifier(n_estimators=50,
                            criterion='entropy')
cl.fit(x_train, y)
random_forest_pred = cl.predict(x_test)
print(pd.DataFrame(random_forest_pred))

     0
0    0
1    0
2    0
3    0
4    1
..  ..
413  0
414  1
415  0
416  0
417  1

[418 rows x 1 columns]


In [31]:
# Support Vector Machine
# Using polynomial kernel
from sklearn.svm import SVC
polyclassifier = SVC(kernel='poly')
polyclassifier.fit(x_train, y)
polypredict = polyclassifier.predict(x_test)

In [34]:
# Support Vector Machine
# Using rbf kernel
rbfclassifier = SVC(kernel='rbf')
rbfclassifier.fit(x_train, y)
rbfpredict = rbfclassifier.predict(x_test)

In [35]:
# Support Vector Machine
# Using sigmoid kernel
sigmoidclassifier = SVC(kernel='sigmoid')
sigmoidclassifier.fit(x_train, y)
sigpredict = sigmoidclassifier.predict(x_test)
print(pd.DataFrame(sigpredict))

     0
0    0
1    0
2    0
3    0
4    0
..  ..
413  0
414  1
415  0
416  0
417  0

[418 rows x 1 columns]


In [ ]:
# In the cell below I go through each model and see what they predicted and 
# fro that I select the prediction that was chosen more.

In [36]:
submission = pd.read_csv("gender_submission.csv")
for i in range(len(sigpredict)):
    one = 0
    zero = 0
    if neural_network_pred[i] == 1:
        one += 1
    else:
        zero += 1
    if polypredict[i] == 1:
        one += 1
    else:
        zero += 1
    if random_forest_pred[i] == 1:
        one += 1
    else:
        zero += 1
    if rbfpredict[i] == 1:
        one += 1
    else:
        zero += 1
    if sigpredict[i] == 1:
        one += 1
    else:
        zero += 1
    if one >= zero:
        submission['Survived'][i] = 1
    else:
        submission['Survived'][i] = 0
        
print(pd.DataFrame(submission))

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [ ]:
# From here I converted the submission array to a csv file and submitted the 
# file onto kaggle and got a score of 0.77511, that is I correctly predicted
# 77% of the targets